In [1]:
import torch
import pandas as pd
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, file_path):
        df = pd.read_csv(file_path)
        self.x1 = df.iloc[:, 0].values
        self.x2 = df.iloc[:, 1].values
        self.y = df.iloc[:, 2].values
        self.length = len(df)

    def __getitem__(self, index):
        x = torch.FloatTensor([self.x1[index], self.x2[index]])
        y = torch.FloatTensor([self.y[index]])
        return x, y
    
    def __len__(self):
        return self.length
    
class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()

        self.layer = nn.Sequential(
            nn.Linear(2, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.layer(x)
        return x

In [7]:
class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()

        self.layer1 = nn.Sequential(
            nn.Linear(2, 2),
            nn.Sigmoid()
        )
        self.layer2 = nn.Sequential(
            nn.Linear(2, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        return x

In [8]:
train_dataset = CustomDataset('./datasets/perceptron.csv')
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, drop_last=True)

In [9]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = CustomModel().to(device)
criterion = nn.BCELoss().to(device)
lr = 0.01
optimizer = optim.SGD(model.parameters(), lr=lr)

In [10]:
epochs = 10000

for epoch in range(epochs):
    cost = 0.0

    for x, y in train_dataloader:
        x = x.to(device)
        y = y.to(device)

        output = model(x)
        loss = criterion(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cost += loss

    cost = cost / len(train_dataloader)

    if (epoch + 1) % 1000 == 0:
        print(f'Epoch : {epoch+1:4d}, Cost : {cost:.3f}')

C:\Users\yjkim\AppData\Local\Temp\ipykernel_36080\4276576919.py:16: DeprecationWarning: In future, it will be an error for 'np.bool' scalars to be interpreted as an index
  x = torch.FloatTensor([self.x1[index], self.x2[index]])
C:\Users\yjkim\AppData\Local\Temp\ipykernel_36080\4276576919.py:17: DeprecationWarning: In future, it will be an error for 'np.bool' scalars to be interpreted as an index
  y = torch.FloatTensor([self.y[index]])


Epoch : 1000, Cost : 0.693
Epoch : 2000, Cost : 0.693
Epoch : 3000, Cost : 0.688
Epoch : 4000, Cost : 0.591
Epoch : 5000, Cost : 0.423
Epoch : 6000, Cost : 0.106
Epoch : 7000, Cost : 0.044
Epoch : 8000, Cost : 0.027
Epoch : 9000, Cost : 0.019
Epoch : 10000, Cost : 0.015


In [11]:
with torch.no_grad():
    model.eval()
    inputs = torch.FloatTensor([
        [0, 0],
        [0, 1],
        [1, 0],
        [1, 1]
    ]).to(device)
    outputs = model(inputs)

    print('---------')
    print(outputs)
    print(outputs <= 0.5)

---------
tensor([[0.0160],
        [0.9866],
        [0.9865],
        [0.0160]], device='cuda:0')
tensor([[ True],
        [False],
        [False],
        [ True]], device='cuda:0')
